In [1]:
# import packages
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

In [11]:
hq_all = pd.read_excel('results/HQcl_all_bins_all_info.xlsx')
lq_all = pd.read_excel('results/LQcl_all_bins_all_info.xlsx')

# concat hq and lq
all_bins = pd.concat([hq_all, lq_all], axis=0).reset_index(drop=True)
all_bins.to_excel('results/ALLcl_all_bins_all_info.xlsx', index=False)

/var/folders/fr/j_4kmkcj6zs6nglntg64jfgr0000gn/T/ipykernel_6070/3257442092.py:6: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.1' currently installed).
  all_bins.to_excel('results/ALLcl_all_bins_all_info.xlsx', index=False)


In [12]:
hq_rep = pd.read_excel('results/HQcl_representative_MAGs_all_info.xlsx')
lq_rep = pd.read_excel('results/LQcl_representative_MAGs_all_info.xlsx')

# concat hq and lq
all_rep = pd.concat([hq_rep, lq_rep], axis=0).reset_index(drop=True)
all_rep.to_excel('results/ALLcl_representative_MAGs_all_info.xlsx', index=False)

/var/folders/fr/j_4kmkcj6zs6nglntg64jfgr0000gn/T/ipykernel_6070/3085213126.py:6: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.1' currently installed).
  all_rep.to_excel('results/ALLcl_representative_MAGs_all_info.xlsx', index=False)
